# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

- We can use the **response_format** argument in the chat completitions method to define the type of response we want. 

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog / news', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'learn hub', 'url': 'https://huggingface.co/learn'},
  {'type': 'docs hub', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discuss forums', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'Endpoints API', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'Blog page', 'url': 'https://huggingface.co/blog'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanVideo-1.5
Updated
about 3 hours ago
•
1.49k
•
634
facebook/sam3
Updated
4 days ago
•
84.2k
•
586
Supertone/supertonic
Updated
3 days ago
•
2.3k
•
220
WeiboAI/VibeThinker-1.5B
Updated
about 10 hours ago
•
15.5k
•
459
facebook/sam-3d-objects
Updated
3 days ago
•
159
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.24k
Qwen Image Edit Camera Control
🎬
1.24k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
CPU Upgrade
Featured
2.4k
The Smol Training Playbook
📚
2.4k
The secrets to building world-class LLMs


In [14]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanVideo-1.5\nUpdated\nabout 3 hours ago\n•\n1.49k\n•\n634\nfacebook/sam3\nUpdated\n4 days ago\n•\n84.2k\n•\n586\nSupertone/supertonic\nUpdated\n3 days ago\n•\n2.3k\n•\n220\nWeiboAI/VibeThinker-1.5B\nUpdated\nabout 10 hours ago\n•\n15.5k\n•\n459\nfacebook/sam-3d-objects\nUpdated\n3 days ago\n•\n159\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\nFeatured\n1.24k\nQwen Image E

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        max_tokens=1024
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the vibrant AI community and collaboration platform shaping the future of machine learning. It serves as a central hub where researchers, developers, and organizations around the world come together to share, explore, and build state-of-the-art AI models, datasets, and applications. 

The platform supports all modalities — text, image, video, audio, and even 3D — enabling users to innovate at the cutting edge of technology. With over 1 million models and 250,000+ datasets, Hugging Face provides a rich ecosystem to accelerate machine learning development and collaboration globally.

---

## What They Offer

- **Hugging Face Hub:** A central repository to host and collaborate on unlimited public models, datasets, and applications.  
- **Spaces:** A popular feature to build and share AI applications in an interactive way, leveraging powerful inference with easy deployment.  
- **Open Source Tools:** Leading open-source libraries and frameworks empower AI engineers to build faster and share better.  
- **Enterprise Solutions:** Scalable, secure AI platform offerings for teams and enterprises, including Single Sign-On, audit logs, granular access controls, advanced compute options, and private storage.  
- **Educational Resources & Community:** Extensive documentation, blogs, forums, and a globally connected community inspire learning and ethical AI development.  

---

## Company Culture

Hugging Face thrives on open collaboration, innovation, and ethical AI development. The community-centric approach means diverse talents and ideas come together to foster an inclusive environment where sharing knowledge is a priority. They are committed to empowering the next generation of machine learning engineers, scientists, and AI practitioners by providing cutting-edge tools and resources.

Their motto: *"Build the future of AI together."* reflects their dedication to a transparent, community-driven, and ethical AI ecosystem.

---

## Customers

Hugging Face’s platform is trusted by:

- Individual machine learning researchers and enthusiasts  
- Startups and independent developers building AI applications  
- Academic institutions advancing AI research  
- Large enterprises requiring robust, scalable AI infrastructure  
- Organizations seeking cutting-edge AI solutions with enterprise-grade security and compliance  

Many leading tech companies and AI innovators rely on Hugging Face for both their open research and production-grade AI deployments.

---

## Careers and Opportunities

Hugging Face offers exciting career opportunities at the forefront of AI technology. They seek passionate individuals who want to:

- Drive advancements in machine learning research  
- Develop open-source frameworks and tools  
- Build scalable AI infrastructure for global impact  
- Shape the future of ethical and open AI  

Interested candidates can explore jobs on their website, joining a fast-growing team known for its vibrant culture, collaborative spirit, and commitment to diversity and inclusion.

---

## Connect & Explore

- **Website:** [huggingface.co](https://huggingface.co)  
- **Community Channels:** GitHub, Twitter, LinkedIn, Discord  
- **Open Source Hub:** Explore millions of models and datasets  
- **Enterprise:** Customized AI solutions with advanced security and support  

---

**Hugging Face** — The AI community building the future, together.

---

*Empower your AI journey with Hugging Face.*  
Sign up today to build, collaborate, and innovate with the world’s leading machine learning platform.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        max_tokens=1024,
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## About Hugging Face
Hugging Face is the vibrant AI community and platform driving the future of machine learning. It serves as the central hub where machine learning engineers, researchers, and enthusiasts can collaborate on models, datasets, applications, and share knowledge to build an open and ethical AI future together.

Positioned as the "Home of Machine Learning," Hugging Face hosts over **1 million models**, **250,000+ datasets**, and **400,000+ applications**, providing tools and infrastructure for all AI modalities including text, image, video, audio, and even 3D data.

---

## Platform & Features

### Collaborative AI Hub
- Host and collaborate on unlimited public models, datasets, and ML applications.
- Share and explore open-source machine learning resources in an active community.
- Build and showcase your ML portfolio to gain community recognition.

### Multi-Modality Support
- Supports diverse AI data types: text, images, videos, audio, and 3D.
- Continuously updated with the latest models from industry-leading organizations and research groups.

### Spaces
- Run interactive ML apps and demos on the platform.
- Access cutting-edge tools for image editing, video generation, depth map extraction, and more.

### Enterprise Solutions
- Tailored Team and Enterprise subscription plans for organizations starting at $20/user/month.
- Enterprise-grade security features include Single Sign-On (SSO), granular access controls, audit logs, and advanced token management.
- Scalable compute options like ZeroGPU with quota boosts.
- Private dataset and storage management.
- Detailed analytics and usage tracking dashboards.

---

## Company Culture
Hugging Face fosters an open, collaborative, and inclusive tech culture focused on:
- **Open-source ethos:** Empowering everyone to contribute and share AI innovations.
- **Community-driven development:** Supporting a fast-growing, passionate global AI community.
- **Ethical AI:** Committed to creating technology that aligns with responsible, transparent AI principles.
- **Innovation at the edge:** Driven by a talented science team exploring new frontiers in AI.

---

## Customers & Community
- Serves a wide range of customers including AI developers, researchers, startups, and large enterprises.
- Trusted by leading AI organizations and integrated with top open-source ML libraries.
- Home to a thriving developer community active in the forum, GitHub, Discord, and social media channels.
- Provides educational and collaborative resources to empower ML learners and professionals worldwide.

---

## Careers at Hugging Face
Hugging Face is continuously growing and seeking talented individuals who are:
- Passionate about machine learning and open-source AI.
- Eager to work in a remote-friendly, innovative environment.
- Motivated to build ethical AI solutions that impact the world positively.

Open positions span engineering, research, data science, product management, and more. Join a mission-driven team at the heart of the AI revolution.

---

## Get Involved
- Explore and contribute to AI models and datasets at [Hugging Face Hub](https://huggingface.co).
- Sign up to build your ML profile and accelerate your projects.
- Engage with the community and stay updated via their GitHub, Twitter, LinkedIn, and Discord channels.
- For enterprise inquiries and tailored solutions, contact their sales team.

---

**Hugging Face**  
_The AI community building the future._

**Website:** https://huggingface.co  
**Social:** GitHub | Twitter | LinkedIn | Discord

---

**Brand Colors:**  
- Yellow: #FFD21E  
- Orange: #FF9D00  
- Gray: #6B7280

---

Join Hugging Face today and be part of shaping the future of machine learning!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>